# BÀI THỰC HÀNH 2: MẠNG NEURAL TÍCH CHẬP

<b>Hướng dẫn nộp bài:</b> Các bạn commit và push code lên github, sử dụng file txt đặt tên theo cú pháp <MSSV>.txt chứa đường link dẫn đến github của bài thực hành và nộp file txt này tên courses.

Bộ dữ liệu sử dụng: [MNIST dataset](https://git-disl.github.io/GTDLBench/datasets/mnist_datasets/) (bài 1) và [VinaFood21 dataset](https://arxiv.org/abs/2108.02929) (các bài còn lại).

Link download: https://drive.google.com/file/d/1UpZOf0XlwvB4rKpyZ35iwTA8oWHqDBbR/view?usp=share_link.

### Bài 1: Xây dựng mô hình LeNet. Huấn luyện và đánh giá mô hình LeNet trên 4 độ đo precision, recall và F1-macro (sử dụng Adam làm optimizer).

![image-2.png](attachment:image-2.png)

![image.png](attachment:image.png)

### Bài 2: Xây dựng mô hình GoogLeNet. Huấn luyện và đánh giá mô hình GoogLeNet trên 4 độ đo precision, recall và F1 (sử dụng Adam làm optimizer). Lưu ý lớp Convolution đầu tiên có padding là 3, các lớp Max Pooling đều bật chế độ ceil_mode (`ceil_mode=True`).

**Inception Blocks**

![image.png](attachment:image.png)

**GoogLeNet (Multi-branch Network)**

![image-2.png](attachment:image-2.png)

**GoogLeNet Parameters**

![image-3.png](attachment:image-3.png)

### Bài 3: Xây dựng mô hình ResNet-18, đánh giá mô hình ResNet-18 trên bộ dữ liệu VinaFood21 sử dụng các độ đo precision, recall, và F1 (Sử dụng Adam làm optimizer). Lưu ý, giữa các block Residual-Connection có một lớp Max Pooling (kernel = 3, stride = 2, paddding = 0)

**ResNet Block**

![image.png](attachment:image.png)

**ResNet**

![image-2.png](attachment:image-2.png)

**ResNet-18 Parameter**
![image-4.png](attachment:image-4.png)

### Bài 4: Sử dụng pretrained ResNet50 từ HuggingFace để fine-tune trên bộ dữ liệu VinaFood21.

# 23521758 Code

## Mount drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# nối đường dẫn để import module trên Colab
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/deep_learning_TH/Lab2')

## Import thu vien

In [4]:
!pip -q install idx2numpy

  Preparing metadata (setup.py) ... done


In [42]:
import importlib #refresh modules
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from mnist_dataset import collate_fn, MNISTDataset
from vinafood_dataset import collate_fn, VinaFood
from GoogLeNet import GoogLeNet
from LeNet import LeNet

In [43]:
# lam moi module, dataset, ...

# vinafood
# import vinafood_dataset
# importlib.reload(vinafood_dataset)
# from vinafood_dataset import collate_fn, VinaFood

# mnist
# import mnist_dataset
# importlib.reload(mnist_dataset)
# from mnist_dataset import collate_fn, MNISTDataset

#lenet
# import LeNet
# importlib.reload(LeNet)
# from LeNet import LeNet

#googlenet
import GoogLeNet
importlib.reload(GoogLeNet)
from GoogLeNet import GoogLeNet

## Tải dataset

In [6]:
!pip install -q gdown

In [7]:
# download mnist
!gdown -q --folder https://drive.google.com/drive/folders/1gvZgA7WNEwBbEQWrYpM6-ySRuXccTzCL?usp=sharing

In [8]:
# download VinaFood21
!gdown -q --id 1UpZOf0XlwvB4rKpyZ35iwTA8oWHqDBbR -O /content/VinaFood21.zip
# unzip
!unzip -q /content/VinaFood21.zip
# remove file zip
!rm /content/VinaFood21.zip

/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(


## MNIST Dataset va DataLoader

In [24]:
train_dataset = MNISTDataset(
    image_path="/content/mnist-dataset/train-images.idx3-ubyte",
    label_path="/content/mnist-dataset/train-labels.idx1-ubyte"
)

test_dataset = MNISTDataset(
    image_path="/content/mnist-dataset/t10k-images.idx3-ubyte",
    label_path="/content/mnist-dataset/t10k-labels.idx1-ubyte"
)

train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=32,
    shuffle=True,
    collate_fn=collate_fn
)

test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=32,
    shuffle=True,
    collate_fn=collate_fn
)

## VinaFood Dataset va DataLoader

In [ ]:
train_dataset = VinaFood(
    path="/content/VinaFood21/train",
    image_size=(224, 224)
)
test_dataset = VinaFood(
    path="/content/VinaFood21/test",
    image_size=(224, 224)
)

train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=32,
    shuffle=True,
    collate_fn=collate_fn
)

test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=32,
    shuffle=True,
    collate_fn=collate_fn
)

## Bai 1: Training LeNet

In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Evaluate
def evaluate(model: nn.Module, dataloader: DataLoader) -> dict[str, float]:
    model.eval()
    predictions = []
    trues = []

    with torch.no_grad():
        for item in dataloader:
            #image: torch.Tensor = item["image"].squeeze(1).to(device).float()
            #label: torch.Tensor = item["label"].to(device).long()

            image: torch.Tensor = item["image"].squeeze(1).to(device)
            label: torch.Tensor = item["label"].to(device)

            output: torch.Tensor = model(image)
            output = torch.argmax(output, dim=1)

            predictions.extend(output.cpu().tolist())
            trues.extend(label.cpu().tolist())

    return{
        "accuracy": accuracy_score(trues, predictions),
        "precision": precision_score(trues, predictions, average="macro"),
        "recall": recall_score(trues, predictions, average="macro"),
        "f1": f1_score(trues, predictions, average="macro")
    }

In [13]:
!mkdir -p "/content/drive/MyDrive/Colab Notebooks/deep_learning_TH/Lab2/checkpoints"

In [14]:
import warnings # bỏ qua các warning
warnings.filterwarnings('ignore')

In [32]:
model1 = LeNet().to(device)
loss_fn1 = nn.CrossEntropyLoss().to(device)
optimizer1 = optim.Adam(model1.parameters(), lr=0.1)

EPOCHS = 10
best_score1 = 0
best_score_name1 = "f1"

for epoch in range(EPOCHS):
    print(f"Epoch {epoch+1}/{EPOCHS}")

    losses = []
    model1.train()

    for item in train_dataloader:
        #image = item["image"].squeeze(1).to(device).float()
        #label = item["label"].to(device).long()

        image = item["image"].to(device)
        label = item["label"].to(device)

        # Forward pass
        output = model1(image)
        loss = loss_fn1(output, label)

        # Backward pass
        optimizer1.zero_grad()
        loss.backward()
        optimizer1.step()

        losses.append(loss.item())

    print(f"Loss: {np.mean(losses):.4f}")

    # Evaluate
    scores = evaluate(model1, test_dataloader)
    for score_name, value in scores.items():
        print(f"{score_name}: {value:.4f}")
    print()

    current_score = scores[best_score_name1]
    if current_score > best_score1:
        best_score1 = current_score
        torch.save(
            model1.state_dict(),
            "/content/drive/MyDrive/Colab Notebooks/deep_learning_TH/Lab2/checkpoints/lenet/best_model.pth"
        )

Epoch 1/10
Loss: 2.4423
accuracy: 0.1135
precision: 0.0114
recall: 0.1000
f1: 0.0204

Epoch 2/10
Loss: 2.4239
accuracy: 0.0982
precision: 0.0098
recall: 0.1000
f1: 0.0179

Epoch 3/10
Loss: 2.4205
accuracy: 0.0958
precision: 0.0096
recall: 0.1000
f1: 0.0175

Epoch 4/10
Loss: 2.4209
accuracy: 0.0982
precision: 0.0098
recall: 0.1000
f1: 0.0179

Epoch 5/10
Loss: 2.4108
accuracy: 0.1028
precision: 0.0103
recall: 0.1000
f1: 0.0186

Epoch 6/10
Loss: 2.4221
accuracy: 0.0958
precision: 0.0096
recall: 0.1000
f1: 0.0175

Epoch 7/10
Loss: 2.4183
accuracy: 0.0974
precision: 0.0097
recall: 0.1000
f1: 0.0178

Epoch 8/10
Loss: 2.4222
accuracy: 0.0980
precision: 0.0098
recall: 0.1000
f1: 0.0179

Epoch 9/10
Loss: 2.4159
accuracy: 0.1135
precision: 0.0114
recall: 0.1000
f1: 0.0204

Epoch 10/10
Loss: 2.4147
accuracy: 0.0958
precision: 0.0096
recall: 0.1000
f1: 0.0175



# Bai 2, 3 DataLoader xây dựng bị crash vì tràn RAM nên dùng DataLoader của TorchVision

## Bai 2: Training GoogLeNet

In [33]:
!pip install torchvision

In [44]:
from torchvision import datasets, transforms

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
BATCH_SIZE = 32
EPOCHS = 30
LR = 0.001

transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

train_dataset = datasets.ImageFolder(
    root='/content/VinaFood21/train',
    transform=transform_train
)
test_dataset  = datasets.ImageFolder(
    root='/content/VinaFood21/test',
    transform=transform_test
)

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2
)
test_loader  = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2
)

In [35]:
def evaluate_model(model, loader, device):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for imgs, labels in loader:
            imgs = imgs.to(device)
            outputs = model(imgs)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            y_true.extend(labels.numpy())
            y_pred.extend(preds)

    precision = precision_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")

In [46]:
model2 = GoogLeNet(num_classes=21).to(device)
loss_fn2 = nn.CrossEntropyLoss()
optimizer2 = optim.Adam(model2.parameters(), lr=LR)

def train_model(model, loader):
    model.train()
    total_loss = 0.0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        optimizer2.zero_grad()
        outputs = model(images)
        loss = loss_fn2(outputs, labels)
        loss.backward()
        optimizer2.step()

        total_loss += loss.item()

    return total_loss / len(loader)

def evaluate_model(model, loader):
    model.eval()
    y_true, y_pred = [], []

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average='macro', zero_division=0)
    rec = recall_score(y_true, y_pred, average='macro', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)

    return acc, prec, rec, f1

In [47]:
for epoch in range(EPOCHS):
    train_loss = train_model(model2, train_loader)
    acc, prec, rec, f1 = evaluate_model(model2, test_loader)

    print(f"Epoch [{epoch+1}/{EPOCHS}]")
    print(f"Train Loss: {train_loss:.4f}")
    print(f"Accuracy: {acc:.4f} | Precision: {prec:.4f} | Recall: {rec:.4f} | F1-score: {f1:.4f}")
    print("-" * 50)

torch.save(model2.state_dict(),
           "/content/drive/MyDrive/Colab Notebooks/deep_learning_TH/Lab2/checkpoints/googlenet")


Epoch [1/30]
Train Loss: 2.6464
Accuracy: 0.2507 | Precision: 0.2016 | Recall: 0.2061 | F1-score: 0.1667
--------------------------------------------------


KeyboardInterrupt: 

## Bai 3: Training ResNet18

In [ ]:
from torchvision import datasets, transforms
from RestNet18 import ResNet18
# === Load dataset VinaFood21 ===

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
BATCH_SIZE = 32
EPOCHS = 10
LR = 0.001

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_dataset = datasets.ImageFolder(
    root='/content/VinaFood21/train',
    transform=transform
)
test_dataset  = datasets.ImageFolder(
    root='/content/VinaFood21/test',
    transform=transform
)

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True
)
test_loader  = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False
)

In [ ]:
# === Khởi tạo model, optimizer, loss ===
model3 = ResNet18(num_classes=21).to(device)
loss_fn3 = nn.CrossEntropyLoss()
optimizer = optim.Adam(model3.parameters(), lr=1e-4)

def train_model(model3, loader):
    model3.train()
    total_loss = 0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model3(imgs)
        loss = loss_fn3(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(loader)

def evaluate_model(model3, loader):
    model3.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model3(imgs)
            preds = torch.argmax(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    acc = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')

    return acc, precision, recall, f1

In [ ]:
for epoch in range(EPOCHS):
    train_loss = train_model(model3, train_loader)
    acc, prec, rec, f1 = evaluate_model(model3, test_loader)

    print(f"Epoch [{epoch+1}/{EPOCHS}]")
    print(f"Train Loss: {train_loss:.4f}")
    print(f"Accuracy: {acc:.4f} | Precision: {prec:.4f} | Recall: {rec:.4f} | F1-score: {f1:.4f}")
    print("-" * 50)

torch.save(model3.state_dict(),
           "/content/drive/MyDrive/Lab2/checkpoints/restnet18")

/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch [1/10]
Train Loss: 2.4719
Accuracy: 0.2436 | Precision: 0.2492 | Recall: 0.2374 | F1-score: 0.2156
--------------------------------------------------


/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch [2/10]
Train Loss: 1.9476
Accuracy: 0.3325 | Precision: 0.3536 | Recall: 0.3212 | F1-score: 0.3175
--------------------------------------------------


/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch [3/10]
Train Loss: 1.6184
Accuracy: 0.3506 | Precision: 0.3688 | Recall: 0.3258 | F1-score: 0.3310
--------------------------------------------------


/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch [4/10]
Train Loss: 1.2704
Accuracy: 0.3707 | Precision: 0.4006 | Recall: 0.3497 | F1-score: 0.3472
--------------------------------------------------


/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


KeyboardInterrupt: 